# enums

> Module containing enums refererenced throughout the project

In [1]:
#| default_exp enums 

In [2]:
#| hide 
%load_ext autoreload
%autoreload 2

In [3]:
#| export 
from enum import Enum, auto

In [4]:
#| export
class SleepOrWake(Enum):
    """
    Very simple enum for fixing, once-and-for-all, meaning of class 0 and 1.
    Use this enum when sensible for clarity in code.
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    SLEEP = 1

    
class SleepStagesWLDR(Enum):
    """
    Similar to SleepStages(Enum), but for Wake/Light/Deep/REM instead of Ns
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    LIGHT = 1
    DEEP = 2
    REM = 3

    def to_masked_sleep_wake(self) -> SleepOrWake:
        match self:
            case SleepStagesWLDR.DO_NOT_USE:
                return SleepOrWake.DO_NOT_USE
            case SleepStagesWLDR.UNSCORED:
                return SleepOrWake.UNSCORED
            case SleepStagesWLDR.WAKE:
                return SleepOrWake.WAKE
            case _:
                return SleepOrWake.SLEEP
    
class SleepStages(Enum):
    """
    Very simple enum for fixing, once-and-for-all, meaning of integer sleep stages.
    Use this enum when possible for clarity in code.
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    N1 = 1
    N2 = 2
    N3 = 3
    N4 = 4
    REM = 5

    def to_masked_sleep_wake(self) -> SleepOrWake:
        return self.to_WLDR().to_masked_sleep_wake()

    def to_WLDR(self) -> SleepStagesWLDR:
        match self:
            case SleepStages.DO_NOT_USE:
                return SleepOrWake.DO_NOT_USE
            case SleepStages.UNSCORED:
                return SleepStagesWLDR.UNSCORED
            case SleepStages.WAKE:
                return SleepStagesWLDR.WAKE
            case SleepStages.N1 | SleepStages.N2:
                return SleepStagesWLDR.LIGHT
            case SleepStages.N3 | SleepStages.N4:
                return SleepStagesWLDR.DEEP
            case SleepStages.REM:
                return SleepStagesWLDR.REM


PSG_Enums = SleepOrWake | SleepStagesWLDR | SleepStages


class SleepClassificationProblem(Enum):
    SLEEP_OR_WAKE = auto()
    SLEEP_STAGES_WLDR = auto()
    SLEEP_STAGES = auto()

    def type_enum(self) -> PSG_Enums:
        match self:
            case SleepClassificationProblem.SLEEP_OR_WAKE:
                return SleepOrWake
            case SleepClassificationProblem.SLEEP_STAGES_WLDR:
                return SleepStagesWLDR
            case SleepClassificationProblem.SLEEP_STAGES:
                return SleepStages


In [5]:
#| export

class KnownFeatures(Enum):
    """
    Expresses to the system which features from data to include.
    This lives here for cyclic import reasons...
    """

    ACTIVITY = auto()
    ACCELEROMETER = auto()
    HEARTRATE = auto()
    PSG = auto()

    # Not settable, derived
    ACCEL_SPECTRO = auto()
    MO_SPECTRO = auto()  # Mads Olsen preprocessing

    @property
    def base_feature(self) -> "KnownFeatures":
        if self.is_accelerometer_based:
            return KnownFeatures.ACCELEROMETER
        elif self.is_activity_based:
            return KnownFeatures.ACTIVITY
        elif self.is_heartrate_based:
            return KnownFeatures.HEARTRATE
        else:
            return self

    @property
    def is_derived(self) -> bool:
        match self:
            case KnownFeatures.ACTIVITY | KnownFeatures.ACCELEROMETER | KnownFeatures.HEARTRATE | KnownFeatures.PSG:
                return False
            case _:
                return True

    @property
    def is_spectral(self) -> bool:
        return self in [KnownFeatures.ACCEL_SPECTRO, KnownFeatures.MO_SPECTRO]

    @property
    def is_accelerometer_based(self) -> bool:
        # Convenience + readability helper
        return self in [
            KnownFeatures.ACCELEROMETER,
            KnownFeatures.ACCEL_SPECTRO,
            KnownFeatures.MO_SPECTRO,
        ]

    @property
    def is_activity_based(self) -> bool:
        return self in [KnownFeatures.ACTIVITY]

    @property
    def is_heartrate_based(self) -> bool:
        return self in [KnownFeatures.HEARTRATE]



In [6]:
#| hide
import nbdev; nbdev.nbdev_export()